# Model Performance - calculation

In [ ]:
#libraries
import pandas as pd
import numpy as np
import os
import sys
import joblib
import warnings
# from datetime import date, datetime
import pickle
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import LeavePGroupsOut
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import pearsonr, spearmanr, norm
import scipy.stats as st
from scipy.stats import zscore as  zscore
from joblib import Parallel, delayed
# from multiprocessing import Manager
from sklearn.cross_decomposition import PLSRegression
import gc
import traceback
import logging
from sklearn.model_selection import KFold
import random

from scipy.stats import gaussian_kde
import matplotlib.ticker as ticker
import matplotlib.colors as mcolors

In [ ]:
# Targets and models
targ_list = ['total_','cryst_','fluid_']
model_list = ['eN2','pls','eN1']

In [ ]:
#directories
root_dir ='/media/hcs-sci-psy-narun/ABCC/fmriresults01/derivatives/ML_Tables/nesi_outputs/std_applicability/'
plot_dir = root_dir + '/prediction_plots/'
tables_path ='/media/hcs-sci-psy-narun/ABCC/fmriresults01/derivatives/ML_Tables/'
if not os.path.isdir(root_dir):
    os.mkdir(root_dir) 
if not os.path.isdir(plot_dir):
    os.mkdir(plot_dir) 

In [ ]:
# run bootstrap on correlations (observed, predicted)
def bootstrap_corr_ci(x, y, n_bootstraps=1000, alpha=0.05):
    boot_corrs = []
    n = len(x)
    
    for _ in range(n_bootstraps):
        # Resample with replacement
        indices = np.random.choice(np.arange(n), size=n, replace=True)
        boot_corrs.append(pearsonr(x[indices], y[indices])[0])
        
    # Confidence intervals from percentiles
    lower = np.percentile(boot_corrs, 100 * (alpha / 2))
    upper = np.percentile(boot_corrs, 100 * (1 - alpha / 2))
    
    return lower, upper

## ABCD

In [ ]:
# ABCD NIH toolbox cognitive scores for each group 
T1 = pd.read_csv(tables_path+'cog_tables/cog_t1.csv', index_col=0)
T1.index = T1.index.str.replace('_', '')
T2 = pd.read_csv(tables_path+'cog_tables/cog_t2.csv', index_col=0)
T2.index = T2.index.str.replace('_', '')
T3 = pd.read_csv(tables_path+'cog_tables/cog_t3.csv', index_col=0)
T3.index = T3.index.str.replace('_', '')
T4 = pd.read_csv(tables_path+'cog_tables/cog_t4.csv', index_col=0)
T4.index = T4.index.str.replace('_', '')
non = pd.read_csv(tables_path+'cog_tables/cog_non.csv', index_col=0)
non.index = non.index.str.replace('_', '')
All = pd.read_csv(tables_path+'cog_tables/cog_all.csv', index_col=0)
All.index = All.index.str.replace('_', '')

In [ ]:
# calculate test performance measures and save them

from scipy.stats import pearsonr
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd


# ABCD predicition performance 
def cal_cor1(group, all_predicted, key, i, col):
    tar = targ[0:4]
    group_pred_ind = list(set(all_predicted.index).intersection(group.dropna().index))
    true_val = yt_all.loc[group_pred_ind].values.flatten()
    pre_val = all_predicted.loc[group_pred_ind].values.flatten()
    p_corr, _ = pearsonr(true_val, pre_val)
    r2 = r2_score(true_val, pre_val)
    rmse = np.sqrt(mean_squared_error(true_val, pre_val))
    mae = mean_absolute_error(true_val, pre_val)
    # prediction_results = prediction_results.append(pd.Series(), ignore_index=True)
    new_row[0] = key
    new_row[col + 1] = round(p_corr, 3)
    new_row[col + 2] = round(r2, 3)
    new_row[col + 3] = round(rmse, 3)
    new_row[col + 4] = round(mae, 3)
    plot_obj = [true_val, pre_val]
    boot_obj = [yt_all.loc[group_pred_ind], all_predicted.loc[group_pred_ind]]

    # Calculate confidence interval for the correlation coefficient
    # fisher_z = np.arctanh(p_corr)
    # se = 1 / np.sqrt(len(group_pred_ind) - 3)
    # z_score = norm.ppf(1 - (1 - 0.95) / 2)
    # ci_lower_z = fisher_z - z_score * se
    # ci_upper_z = fisher_z + z_score * se
    # ci_lower = np.tanh(ci_lower_z)
    # ci_upper = np.tanh(ci_upper_z)
    # Calculate bootstrap confidence intervals
    ci_lower, ci_upper = bootstrap_corr_ci(true_val, pre_val)

    # plt.show()
    plt.close('all')
    return [plot_obj, r2, p_corr, len(group_pred_ind), boot_obj, [ci_lower, ci_upper]]

In [ ]:
# Main: Single - ABCD - ABCC
#Load all test folds, calculate predicitve performance, save.
root_dir = '/media/hcs-sci-psy-narun/ABCC/fmriresults01/derivatives/ML_Tables/nesi_outputs/std_applicability/'
targ_list = ['cryst_','fluid_']#,,'cryst_','fluid_''total_'
model_list = ['mnct_enet1'] #'eN1', 'pls','pls','abcd_eN1', 'abcd_pls''eN1',, 'arcn_pls'
for model in model_list:
      for targ in targ_list:
            model_outs = joblib.load(root_dir + targ + model + '.joblib')
            # model_outs = joblib.load(root_dir + targ + 'abcd_' + model + '_std.joblib')
            
            prediction_results = pd.DataFrame(columns=['', 'Tier1','Tier1','Tier1','Tier1','Tier2','Tier2','Tier2','Tier2',
                                                                                                      'Tier3', 'Tier3', 'Tier3','Tier3', 'Tier4','Tier4', 'Tier4','Tier4', 
                                                                                                      'non-ADHD','non-ADHD','non-ADHD','non-ADHD', 'All', 'All','All','All'])
            measure_list = ['Models','r', 'R2', 'MAE', 'RMSE', 'r', 'R2', 'MAE', 'RMSE', 'r', 'R2', 'MAE', 'RMSE','r', 'R2', 'MAE', 'RMSE','r', 'R2', 'MAE', 'RMSE','r', 'R2', 'MAE', 'RMSE']
            prediction_results.loc[0] = measure_list
            moda_yps = {'smri': {}, 'conn': {}, 'cntr': {}, 'gtfc': {} }#, 'tang': {}
            moda_yts = {'smri': {}, 'conn': {}, 'cntr': {}, 'gtfc': {}}#, 'tang': {}
            corr_results = {'smri': {}, 'conn': {}, 'cntr': {}, 'gtfc': {} }#, 'tang': {}
            # moda_yps = {'abcd_rsmri': {}, 'abcd_cntr': {} }#, 'tang': {}
            # moda_yts = {'abcd_rsmri': {}, 'abcd_cntr': {} }#, 'tang': {}
            # corr_results = {'abcd_rsmri': {}, 'abcd_cntr': {} }#, 'tang': {}
            plot_dict = {}
            for fold, sets_dict in model_outs.items():
                    for set_n, set_dict in sets_dict.items():
                        for feature_n, feature_dict in set_dict.items():
                                if feature_n not in moda_yps[set_n]:
                                    moda_yps[set_n][feature_n] = []
                                    moda_yts[set_n][feature_n] = []
                                moda_yps[set_n][feature_n].append(feature_dict['data']['yptest'])
                                moda_yts[set_n][feature_n].append(feature_dict['data']['yttest'])

            for set_name , moda in moda_yps.items():
                for iter, (f_name, y) in enumerate(moda.items()):
                        yp_all = pd.concat(y, axis=0)
                        yt_all = pd.concat(moda_yts[set_name][f_name], axis=0)
                        new_row = pd.Series([None] * len(prediction_results.columns), index=prediction_results.columns)
                        corr_results[set_name][f_name] = {}
                        corr_results[set_name][f_name]['Tier1'] = cal_cor1(T1, yp_all, f_name, iter+1, 0)
                        corr_results[set_name][f_name]['Tier2']  = cal_cor1(T2, yp_all, f_name, iter+1, 4)
                        corr_results[set_name][f_name]['Tier3']  = cal_cor1( T3, yp_all, f_name, iter+1,  8)
                        corr_results[set_name][f_name]['Tier4']  = cal_cor1(T4, yp_all, f_name, iter+1,  12)
                        corr_results[set_name][f_name]['non-ADHD']  = cal_cor1(non, yp_all, f_name, iter+1, 16)
                        corr_results[set_name][f_name]['All']  = cal_cor1(All, yp_all, f_name, iter+1, 20)
                        reshaped_row = new_row.values.reshape(1, -1)  # Reshape to a single row
                        reshaped_row_df = pd.DataFrame(reshaped_row, columns=prediction_results.columns)
                        prediction_results = pd.concat([prediction_results, reshaped_row_df], ignore_index=True)
                        # prediction_results = prediction_results.append(new_row, ignore_index=True)
                        #pd.concat([prediction_results, pd.DataFrame([new_row], columns=prediction_results.columns)], ignore_index=True)
            prediction_results.to_csv(plot_dir  + targ + model + '_predictive_ability.csv')
            joblib.dump(corr_results, plot_dir  + targ + model + '_corr_res.joblib')
            

In [ ]:
# Main: Single - ABCD - ABCD

root_dir = '/media/hcs-sci-psy-narun/ABCC/fmriresults01/derivatives/ML_Tables/nesi_outputs/std_applicability/'
targ_list = ['total_','cryst_','fluid_']#,,'cryst_','fluid_'
model_list = ['eN1', 'pls'] #'eN1', 'pls','pls','abcd_eN1', 'abcd_pls'
for model in model_list:
      for targ in targ_list:
            # model_outs = joblib.load(root_dir + targ + model + '_std.joblib')
            model_outs = joblib.load(root_dir + targ + 'abcd_' + model + '_std.joblib')
            
            prediction_results = pd.DataFrame(columns=['', 'Tier1','Tier1','Tier1','Tier1','Tier2','Tier2','Tier2','Tier2',
                                                                                                      'Tier3', 'Tier3', 'Tier3','Tier3', 'Tier4','Tier4', 'Tier4','Tier4', 
                                                                                                      'non-ADHD','non-ADHD','non-ADHD','non-ADHD', 'All', 'All','All','All'])
            measure_list = ['Models','r', 'R2', 'MAE', 'RMSE', 'r', 'R2', 'MAE', 'RMSE', 'r', 'R2', 'MAE', 'RMSE','r', 'R2', 'MAE', 'RMSE','r', 'R2', 'MAE', 'RMSE','r', 'R2', 'MAE', 'RMSE']
            prediction_results.loc[0] = measure_list
            # moda_yps = {'smri': {}, 'conn': {}, 'cntr': {}, 'gtfc': {} }#, 'tang': {}
            # moda_yts = {'smri': {}, 'conn': {}, 'cntr': {}, 'gtfc': {}}#, 'tang': {}
            # corr_results = {'smri': {}, 'conn': {}, 'cntr': {}, 'gtfc': {} }#, 'tang': {}
            moda_yps = {'abcd_rsmri': {}, 'abcd_cntr': {} }#, 'tang': {}
            moda_yts = {'abcd_rsmri': {}, 'abcd_cntr': {} }#, 'tang': {}
            corr_results = {'abcd_rsmri': {}, 'abcd_cntr': {} }#, 'tang': {}
            plot_dict = {}
            for fold, sets_dict in model_outs.items():
                    for set_n, set_dict in sets_dict.items():
                        for feature_n, feature_dict in set_dict.items():
                                if feature_n not in moda_yps[set_n]:
                                    moda_yps[set_n][feature_n] = []
                                    moda_yts[set_n][feature_n] = []
                                moda_yps[set_n][feature_n].append(feature_dict['data']['yptest'])
                                moda_yts[set_n][feature_n].append(feature_dict['data']['yttest'])

            for set_name , moda in moda_yps.items():
                for iter, (f_name, y) in enumerate(moda.items()):
                        yp_all = pd.concat(y, axis=0)
                        yt_all = pd.concat(moda_yts[set_name][f_name], axis=0)
                        new_row = pd.Series([None] * len(prediction_results.columns), index=prediction_results.columns)
                        corr_results[set_name][f_name] = {}
                        corr_results[set_name][f_name]['Tier1'] = cal_cor1(T1, yp_all, f_name, iter+1, 0)
                        corr_results[set_name][f_name]['Tier2']  = cal_cor1(T2, yp_all, f_name, iter+1, 4)
                        corr_results[set_name][f_name]['Tier3']  = cal_cor1( T3, yp_all, f_name, iter+1,  8)
                        corr_results[set_name][f_name]['Tier4']  = cal_cor1(T4, yp_all, f_name, iter+1,  12)
                        corr_results[set_name][f_name]['non-ADHD']  = cal_cor1(non, yp_all, f_name, iter+1, 16)
                        corr_results[set_name][f_name]['All']  = cal_cor1(All, yp_all, f_name, iter+1, 20)
                        # prediction_results = prediction_results.append(new_row, ignore_index=True)
                        reshaped_row = new_row.values.reshape(1, -1)  # Reshape to a single row
                        reshaped_row_df = pd.DataFrame(reshaped_row, columns=prediction_results.columns)
                        prediction_results = pd.concat([prediction_results, reshaped_row_df], ignore_index=True)
                        #pd.concat([prediction_results, pd.DataFrame([new_row], columns=prediction_results.columns)], ignore_index=True)
            prediction_results.to_csv(plot_dir  + targ + model + '_abcd_predictive_ability.csv')
            joblib.dump(corr_results, plot_dir  + targ + model + '_abcd_corr_res.joblib')
            

In [ ]:
# Main: Stacked - ABCD + ABCC

root_dir = '/media/hcs-sci-psy-narun/ABCC/fmriresults01/derivatives/ML_Tables/nesi_outputs/std_applicability/'

targ_list = ['total_', 'cryst_','fluid_'] #'total_','fluid_'','fluid_', 'cryst_''total_'

model_list = ['ntrf2']#,'pls','eN1''rf2''eN2',
for model in model_list:
      for targ in targ_list:
            # model_outs = joblib.load(root_dir + targ + model + '.joblib')
            # with open(root_dir + targ + model + 'rf2_output.pkl', 'rb') as f:
            #       model_outs = pickle.load(f)
            model_outs = joblib.load(root_dir + targ + model + '.joblib')
            prediction_results = pd.DataFrame(columns=['', 'Tier1','Tier1','Tier1','Tier1','Tier2','Tier2','Tier2','Tier2',
                                                                                                      'Tier3', 'Tier3', 'Tier3','Tier3', 'Tier4','Tier4', 'Tier4','Tier4', 
                                                                                                      'non-ADHD','non-ADHD','non-ADHD','non-ADHD', 'All', 'All','All','All'])
            measure_list = ['Models','r', 'R2', 'MAE', 'RMSE', 'r', 'R2', 'MAE', 'RMSE', 'r', 'R2', 'MAE', 'RMSE','r', 'R2', 'MAE', 'RMSE','r', 'R2', 'MAE', 'RMSE','r', 'R2', 'MAE', 'RMSE']
            prediction_results.loc[0] = measure_list
            # r2_results = pd.DataFrame(['Tier1','Tier2', 'Tier3', 'Tier4', 'non-ADHD', 'All'], columns=['group'])
            moda_yps = {'stacked': {}}
            moda_yts = {'stacked': {}}
            corr_results = {'stacked': {}}
            plot_dict = {}
            for fold, sets_dict in model_outs.items():
                  for set_n, set_dict in sets_dict.items():
                        for feature_n, feature_dict in set_dict.items():
                              if feature_n not in moda_yps[set_n]:
                                    moda_yps[set_n][feature_n] = []
                                    moda_yts[set_n][feature_n] = []
                              moda_yps[set_n][feature_n].append(feature_dict['data']['yptest'])
                              moda_yts[set_n][feature_n].append(feature_dict['data']['yttest'])

            for set_name , moda in moda_yps.items():
                  for iter, (f_name, y) in enumerate(moda.items()):
                        yp_all = pd.concat(y, axis=0)
                        yt_all = pd.concat(moda_yts[set_name][f_name], axis=0)
                        new_row = pd.Series([None] * len(prediction_results.columns), index=prediction_results.columns)
                        corr_results[set_name][f_name] = {}
                        corr_results[set_name][f_name]['Tier1'] = cal_cor1(T1, yp_all, f_name, iter+1, 0)
                        corr_results[set_name][f_name]['Tier2']  = cal_cor1(T2, yp_all, f_name, iter+1, 4)
                        corr_results[set_name][f_name]['Tier3']  = cal_cor1( T3, yp_all, f_name, iter+1,  8)
                        corr_results[set_name][f_name]['Tier4']  = cal_cor1(T4, yp_all, f_name, iter+1,  12)
                        corr_results[set_name][f_name]['non-ADHD']  = cal_cor1(non, yp_all, f_name, iter+1, 16)
                        corr_results[set_name][f_name]['All']  = cal_cor1(All, yp_all, f_name, iter+1, 20)
                        
                        reshaped_row = new_row.values.reshape(1, -1)  # Reshape to a single row
                        reshaped_row_df = pd.DataFrame(reshaped_row, columns=prediction_results.columns)
                        prediction_results = pd.concat([prediction_results, reshaped_row_df], ignore_index=True)
                        # prediction_results = prediction_results.append(new_row, ignore_index=True)
                        pred_true = pd.concat([yp_all,yt_all], axis=1) 
                        #pred_true.to_csv(plot_dir  + targ + model + f_name +'_predictive_ability.csv')
            prediction_results.to_csv(plot_dir  + targ + model + '_predictive_ability.csv')
            joblib.dump(corr_results, plot_dir  + targ + model + '_corr_res.joblib')



## Lytle

In [ ]:
#WISC-II cognitive scores for each group 
adhd_cog2 = pd.read_csv('/media/hcs-sci-psy-narun/OpenNEURO_adhd/ds003500-download/derivatives/ML_Tables/cog_adhd2.csv', index_col=0)
non_cog2 = pd.read_csv('/media/hcs-sci-psy-narun/OpenNEURO_adhd/ds003500-download/derivatives/ML_Tables/cog_non2.csv', index_col=0)
all_cog2 = pd.read_csv('/media/hcs-sci-psy-narun/OpenNEURO_adhd/ds003500-download/derivatives/ML_Tables/cog.csv', index_col=0)

adhd_cog2.index = adhd_cog2.index.str.replace('sub-', '')
non_cog2.index = non_cog2.index.str.replace('sub-', '')
all_cog2.index = all_cog2.index.str.replace('sub-', '')

adhd_cog1 = pd.read_csv('/media/hcs-sci-psy-narun/OpenNEURO_adhd/ds002424-download/derivatives/ML_Tables/cog_adhd1.csv', index_col=0)
non_cog1 = pd.read_csv('/media/hcs-sci-psy-narun/OpenNEURO_adhd/ds002424-download/derivatives/ML_Tables/cog_non1.csv', index_col=0)
all_cog1 = pd.read_csv('/media/hcs-sci-psy-narun/OpenNEURO_adhd/ds002424-download/derivatives/ML_Tables/cog.csv', index_col=0)
adhd_cog1.index = adhd_cog1.index.str.replace('sub-', '')
non_cog1.index = non_cog1.index.str.replace('sub-', '')
all_cog1.index = all_cog1.index.str.replace('sub-', '')

In [ ]:
# calculate test performance measures and save them

def cal_cor_t(group, all_predicted, key, i, col):
    tar = targ[0:3]
    group_pred_ind = list(set(all_predicted.index).intersection(group.dropna().index))

    # true_val = group.loc[group_pred_ind].filter(like=tar, axis=1).values.flatten()
    # yt_scaler = StandardScaler()
    # yt_te_r = yt_all.values.reshape(-1, 1)
    # syt_all = pd.DataFrame(y_scaler.transform(yt_te_r),  index=yt_all.index)
    strue_val = yt_all.loc[group_pred_ind].values.flatten()
    pre_val = all_predicted.loc[group_pred_ind].values.flatten()


    p_corr, p_val = pearsonr(strue_val, pre_val)
    r2 = r2_score(strue_val, pre_val)
    # print(group.loc[group_pred_ind].filter(like=tar, axis=1).describe())
    rmse = np.sqrt(mean_squared_error(strue_val, pre_val))
    mae = mean_absolute_error(strue_val, pre_val)
    # prediction_results = prediction_results.append(pd.Series(), ignore_index=True)
    new_row[0] = key
    new_row[col + 1] = round(p_corr, 3)
    new_row[col + 2] = round(r2, 3)
    new_row[col + 3] = round(rmse, 3)
    new_row[col + 4] = round(mae, 3)
    # plot_obj = plt.scatter(strue_val, pre_val)
    plot_obj = [strue_val , pre_val]
    boot_obj = [yt_all.loc[group_pred_ind], all_predicted.loc[group_pred_ind]]
        # Calculate confidence interval for the correlation coefficient
    # fisher_z = np.arctanh(p_corr)
    # se = 1 / np.sqrt(len(group_pred_ind) - 3)
    # z_score = norm.ppf(1 - (1 - 0.95) / 2)
    # ci_lower_z = fisher_z - z_score * se
    # ci_upper_z = fisher_z + z_score * se
    # ci_lower = np.tanh(ci_lower_z)
    # ci_upper = np.tanh(ci_upper_z)
    ci_lower, ci_upper = bootstrap_corr_ci(strue_val, pre_val)

    # plt.show()
    plt.close('all')
    return [plot_obj, r2, p_corr, len(group_pred_ind), boot_obj, [ci_lower, ci_upper]]


In [ ]:
# Main: Single - Lytle

T22 =  pd.DataFrame()
T32 =  pd.DataFrame()
T42 =  pd.DataFrame()
root_dir = '/media/hcs-sci-psy-narun/ABCC/fmriresults01/derivatives/ML_Tables/nesi_outputs/std_applicability/'
targ_list = ['VIQ_', 'PIQ_', 'FSIQ_']
model_list = ['enet1_output_fstd'] #'eN1', 'pls','pls',, 'pls1_output_fstd'
test_set = '_op1'#

for model in model_list:
      for targ in targ_list:
            print(targ)
            model_outs = joblib.load(root_dir + targ +  model + test_set + '.joblib')

            prediction_results = pd.DataFrame(columns=['Models', 'Tier1','Tier1','Tier1','Tier1','non-ADHD','non-ADHD','non-ADHD','non-ADHD', 'All', 'All','All','All'])
            measure_list = ['Models','r', 'R2', 'MAE', 'RMSE', 'r', 'R2', 'MAE', 'RMSE', 'r', 'R2', 'MAE', 'RMSE']
            prediction_results.loc[0] = measure_list
            moda_yps = {'smri': {}, 'conn': {}, 'gtfc': {} ,'cntr': {}}
            moda_yts = {'smri': {}, 'conn': {}, 'gtfc': {} ,'cntr': {}}
            corr_results = {'smri': {}, 'conn': {}, 'gtfc': {} ,'cntr': {}}
            plot_dict = {}
            folds_results = {}
            # for fold, sets_dict in model_outs.items():
            nsets_dict= {
                'smri': {key: model_outs[key] for key in ['cort', 'surf', 'subc', 'VolBrain']},
                'conn': {key: model_outs[key] for key in ['verbal', 'spatial']},#
                'gtfc': {key: model_outs[key] for key in ['tfc', 'gtfc']},
                'cntr': {key: model_outs[key] for key in ['cntr_twoback_task-verbal', 'cntr_oneback_task-verbal', 'cntr_twoback-oneback_task-verbal', 'cntr_twoback_task-spatial', 
                                                            'cntr_oneback_task-spatial', 'cntr_twoback-oneback_task-spatial']} #
                }

            for set_n, set_dict in nsets_dict.items():
                for  iter, (f_name, feature_dict) in enumerate(set_dict.items()):
                        if f_name not in moda_yps[set_n]:
                            moda_yps[set_n][f_name] = []
                            moda_yts[set_n][f_name] = []
                        yp_all = feature_dict['data']['yptest']
                        yt_all = feature_dict['data']['yttest']
                        #print(yp_all, yt_all)

                        new_row = pd.Series([None] * len(prediction_results.columns), index=prediction_results.columns)
                        corr_results[set_n][f_name] = {}
                        corr_results[set_n][f_name]['ADHD'] = cal_cor_t(adhd_cog1, yp_all, f_name, iter+1, 0)

                        corr_results[set_n][f_name]['non-ADHD']  = cal_cor_t(non_cog1, yp_all, f_name, iter+1, 4)
                        corr_results[set_n][f_name]['All']  = cal_cor_t(all_cog1, yp_all, f_name, iter+1, 8)
                        # prediction_results = prediction_results.append(new_row, ignore_index=True)

                        reshaped_row = new_row.values.reshape(1, -1)  # Reshape to a single row
                        reshaped_row_df = pd.DataFrame(reshaped_row, columns=prediction_results.columns)
                        prediction_results = pd.concat([prediction_results, reshaped_row_df], ignore_index=True)
                        # prediction_results = prediction_results.append(new_row, ignore_index=True)
                        pred_true = pd.concat([yp_all,yt_all], axis=1) 
                        #pred_true.to_csv(plot_dir  + targ + model + f_name +'_predictive_ability.csv')

            # prediction_results_grouped = prediction_results.loc[1:,:].groupby('Models').mean()
            # prediction_results_grouped = pd.concat([new_row, prediction_results_grouped], ignore_index=True)
            prediction_results.to_csv(plot_dir  + targ + model + test_set + '_predictive_ability.csv')
            joblib.dump(corr_results, plot_dir  + targ + model + test_set + '_corr_res.joblib')

In [ ]:
# Main: Stacked - Lytle

adhd_cog = pd.read_csv('/media/hcs-sci-psy-narun/OpenNEURO_adhd/ds002424-download/derivatives/ML_Tables/cog_adhd1.csv', index_col=0)
non_cog = pd.read_csv('/media/hcs-sci-psy-narun/OpenNEURO_adhd/ds002424-download/derivatives/ML_Tables/cog_non1.csv', index_col=0)
all_cog = pd.read_csv('/media/hcs-sci-psy-narun/OpenNEURO_adhd/ds002424-download/derivatives/ML_Tables/cog.csv', index_col=0)
adhd_cog.index = adhd_cog.index.str.replace('sub-', '')
non_cog.index = non_cog.index.str.replace('sub-', '')
all_cog.index = all_cog.index.str.replace('sub-', '')
T22 =  pd.DataFrame()
T32 =  pd.DataFrame()
T42 =  pd.DataFrame()
root_dir = '/media/hcs-sci-psy-narun/ABCC/fmriresults01/derivatives/ML_Tables/nesi_outputs/std_applicability/'
targ_list = ['VIQ_', 'PIQ_', 'FSIQ_']
test_set = '_op1' #'op2_'
model_list = ['rf2_output_fstd']#

for model in model_list:
      for targ in targ_list:

            model_outs = joblib.load(root_dir + targ +  model + test_set + '.joblib')
            prediction_results = pd.DataFrame(columns=['Models', 'Tier1','Tier1','Tier1','Tier1','non-ADHD','non-ADHD','non-ADHD','non-ADHD', 'All', 'All','All','All'])
            measure_list = ['Model','r', 'R2', 'MAE', 'RMSE', 'r', 'R2', 'MAE', 'RMSE', 'r', 'R2', 'MAE', 'RMSE']
            prediction_results.loc[0] = measure_list
           
            moda_yps = {'stacked': {}}
            moda_yts = {'stacked': {}}
            corr_results = {'stacked': {}}
            plot_dict = {}
            folds_results = {}
            # for fold, sets_dict in model_outs.items():
            nmodel_outs = {'stacked': model_outs}       
            for set_n, set_dict in nmodel_outs.items():
                for  iter, (f_name, feature_dict) in enumerate(set_dict.items()):
                        if f_name not in moda_yps[set_n]:
                            moda_yps[set_n][f_name] = []
                            moda_yts[set_n][f_name] = []
                        yp_all = feature_dict['data']['yptest']
                        yt_all = feature_dict['data']['yttest']
                        #print(yp_all)
                        new_row = pd.Series([None] * len(prediction_results.columns), index=prediction_results.columns)
                        corr_results[set_n][f_name] = {}
                        corr_results[set_n][f_name]['ADHD'] = cal_cor_t(adhd_cog, yp_all, f_name, iter+1, 0)

                        corr_results[set_n][f_name]['non-ADHD']  = cal_cor_t(non_cog, yp_all, f_name, iter+1, 4)
                        corr_results[set_n][f_name]['All']  = cal_cor_t(all_cog, yp_all, f_name, iter+1, 8)
                        reshaped_row = new_row.values.reshape(1, -1)  # Reshape to a single row
                        reshaped_row_df = pd.DataFrame(reshaped_row, columns=prediction_results.columns)
                        prediction_results = pd.concat([prediction_results, reshaped_row_df], ignore_index=True)

            # prediction_results_grouped = prediction_results.loc[1:,:].groupby('Models').mean()
            # measure_list = pd.DataFrame([measure_list], columns=prediction_results.columns)
            # prediction_results_grouped = pd.concat([measure_list, prediction_results_grouped], ignore_index=True)
            prediction_results.to_csv(plot_dir + targ + model + test_set +  '_predictive_ability.csv')
            joblib.dump(corr_results, plot_dir  + targ + model + test_set + '_corr_res.joblib')